# Differential expression analysis using decoupler

This workbook performs differential expression analysis at pseudobulk level using decoupler. 

In [ ]:
#import besca as bc
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from scipy import sparse, io
import os
import time
import logging
import pkg_resources
import seaborn as sns
import itertools
import sys

# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
#from pydeseq2.default_inference import DefaultInference

sc.logging.print_versions()

# for standard processing, set verbosity to minimum
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)
version = '2.4'
start0 = time.time()

In [ ]:
#define standardized filepaths based on above input
root_path = os.getcwd()
#bescapath_full = os.path.dirname(bc.__file__)
#bescapath = os.path.split(bescapath_full)[0]

analysis_name = 'sw_besca24'
species='mouse' ## or mouse for now

results_folder = os.path.join(root_path, 'analyzed/',analysis_name)
results_file = os.path.join(results_folder, analysis_name + '.annotated.h5ad')
results_file_raw = os.path.join(results_folder, analysis_name + '.raw.h5ad')
results_folder_out = results_folder+ '/DE/sc_decoupler/'
#results_folder_pseudo = results_folder+ '/DE/Pseudobulk/'
#results_folder_pseudo = results_folder+ '/DE/PseudobulkRaw/'

clusters='leiden'
split_condition='readout_id' #'experiment' is generally a good default ### change to sampleID

In [ ]:
import decoupler as dc
# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [ ]:
adata = sc.read_h5ad(results_file)
adataraw=sc.read_h5ad(results_file_raw)


In [ ]:
adata.uns['log1p'] = {'base' : None} # Fix for bug related to scanpy version scverse/scanpy#2239

In [ ]:
# Store raw counts in layers
adataraw.X = np.round(adataraw.X)
adataraw.layers['counts'] = adataraw.X


In [ ]:
#adataraw.layers['normalized']=adata.raw.X.copy()

In [ ]:
adataraw=adataraw[adata.obs.index].copy()

In [ ]:
#adataraw=adataraw[adata.obs.index,adata.var.index].copy()

In [ ]:
#adataraw.X=adata.raw.X.copy()

In [ ]:
#adataraw.var=adata.var.copy()
adataraw.obs=adata.obs.copy()

In [ ]:
adataraw.obsm=adata.obsm.copy()
adataraw.obsp=adata.obsp.copy()

In [ ]:
sc.pl.umap(adataraw,color='celltype1')

In [ ]:
#adataraw=adataraw[adataraw.obs['celltype1']!='artifact'].copy()

In [ ]:
sc.pl.umap(adataraw,color='celltype2')

In [ ]:
sc.pl.umap(adataraw,color='celltype0')

In [ ]:
#sc.pl.umap(cdata,color='celltype2')

In [ ]:
sc.settings.set_figure_params()

#### Select some genes of interest to follow

In [ ]:
### these gois are from the pseudobulk DE analysis
goiinf=['Ifng','Tnf','Il12a','Il12b','Cxcl9','Cxcl10','Ccl2','Ccl3','Ccl4','Il6','Il1b',
       'Nos1','Nos2','Nos3','Cd86','Cd14','Il1r1','Csf2','Slc2a1','Arg1','Folr2','Spp1'] # E.g: ['PTPN2', 'PTPN22']
goirep=['Cd274','Pdcd1lg2','Ido1','Arg1','Tgfb1','Il1b','Il4','Il6','Il10','Il13','Ccl17',
         'Ccl22','Vegfa','Pdgfa','Egf','Hgf','Csf1',
       'Inhba','Gpnmb','Mmp2','Mmp9','Mmp13','Mmp14',
          'Ctsb','Ctsd','Ctsk','Ctss','Cd14','Fcgr3',
       'Csf1r','Mrc1','Cd36','Ccr2']


In [ ]:
### these gois are from the pseudobulk DE analysis
goiorder=['Ifng','Tnf','Il12a','Il12b','Cxcl9','Cxcl10','Ccl2','Ccl3','Ccl4','Il6','Il1b',
       'Nos1','Nos2','Nos3','Cd86','Cd14','Il1r1','Csf2','Slc2a1','Arg1','Folr2','Spp1', # E.g: ['PTPN2', 'PTPN22']
'Cd274','Pdcd1lg2','Ido1','Arg1','Tgfb1','Il1b','Il4','Il6','Il10','Il13','Ccl17',
         'Ccl22','Vegfa','Pdgfa','Egf','Hgf','Csf1',
       'Inhba','Gpnmb','Mmp2','Mmp9','Mmp13','Mmp14',
          'Ctsb','Ctsd','Ctsk','Ctss','Cd14','Fcgr3',
       'Csf1r','Mrc1','Cd36','Ccr2']

goi=goiorder
#goi=[x.upper() for x in goi]
sc.pl.dotplot(adata,var_names=goi,groupby='celltype4',dot_max=0.8,vmax=2)


In [ ]:
sc.pl.dotplot(adata,var_names=goi,groupby='treatment_id', 
              dot_max=0.4, vmax=1.5)


In [ ]:
#set(adata.obs['MYCOND'])

#### Create conditions used for pairwise comparisons

In [ ]:

#ax=ups.loc[list(set(goiorder).intersection(set(ups.index))),:].sort_values(by='log2FoldChange')['log2FoldChange'].plot.bar()
#ax.set_ylabel("log2FoldChange")
sc.settings.set_figure_params()


adataraw.obs['MYCOND']=adataraw.obs.CONDITION.astype(str)
adata.obs['MYCOND']=adata.obs.CONDITION.astype(str)

adataraw.obs['MYCOND'].replace('C_t0', 'T0', inplace=True)
adataraw.obs['MYCOND'].replace('T_1', 'T1', inplace=True)
adataraw.obs['MYCOND'].replace('T_2', 'T2', inplace=True)
adataraw.obs['MYCOND'].replace('T_t3', 'T3', inplace=True)
adataraw.obs['MYCOND'].replace('T_t4', 'T4', inplace=True)
adataraw.obs['MYCOND'].replace('T_t5', 'T5', inplace=True)

adata.obs['MYCOND'].replace('C_t0', 'T0', inplace=True)
adata.obs['MYCOND'].replace('T_1', 'T1', inplace=True)
adata.obs['MYCOND'].replace('T_2', 'T2', inplace=True)
adata.obs['MYCOND'].replace('T_t3', 'T3', inplace=True)
adata.obs['MYCOND'].replace('T_t4', 'T4', inplace=True)
adata.obs['MYCOND'].replace('T_t5', 'T5', inplace=True)


sc.pl.umap(adataraw, color='MYCOND')

sc.pl.umap(adata, color='MYCOND')


In [ ]:
#adata.obs['MYCOND']=adata.obs.sex.astype(str)+adata.obs.CONDITION.astype(str)


In [ ]:
sc.settings.set_figure_params()

#### Make DE per subset of cell

In [ ]:
sc.pl.umap(adata, color='CONDITION')

In [ ]:
mysubset=['macrophage']

mylevel="celltype2"
cdata=adataraw[adataraw.obs[mylevel].isin(mysubset)].copy()
cdatak=adata[adata.obs[mylevel].isin(mysubset)].copy()


mysubset='Macrophage'


condition='MYCOND'

results_out_cell=results_folder_out+'/'+mysubset.replace(" ", "_")+'/'
if not os.path.exists(results_out_cell): os.mkdir(results_out_cell)
if not os.path.exists(results_out_cell+'enrichr/'): os.mkdir(results_out_cell+'enrichr/')

    

split_condition='readout_id'

In [ ]:

### Remove 2 samples from analysis - 
cdata=cdata.copy() #[~cdata.obs.readout_id.isin(['8050P21_352_rep2','8050P21_201'])].copy()
### Remove 2 samples from analysis - 
cdatak=cdatak.copy() #[~cdatak.obs.readout_id.isin(['8050P21_352_rep2','8050P21_201'])].copy()

#### Plot genes of interest

mypval=0.05
myfc=np.log2(1.5)

condition

conds=list(cdata.obs[condition].cat.categories)

conds

### Could also check pairwise comparisons if needed
AllComparisons=list(itertools.combinations(conds,2))

AllComparisons


In [ ]:


mypairs=[AllComparisons[0],AllComparisons[1],AllComparisons[2], AllComparisons[3], AllComparisons[4]
        , AllComparisons[5], AllComparisons[9], AllComparisons[12], AllComparisons[14]]

mypairs

In [ ]:
# Get pseudo-bulk profile
pdata = dc.get_pseudobulk(cdata,
    sample_col=split_condition,
    groups_col='MYCOND',
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=50
)
pdata

In [ ]:
dc.plot_psbulk_samples(pdata, groupby=['compound_name', 'CONDITION'], figsize=(11, 3))

In [ ]:
#### Normalise data 
pp_data=pdata.copy()
sc.pp.normalize_total(pp_data, target_sum=1e6)
sc.pp.log1p(pp_data)
sc.pp.scale(pp_data, max_value=10)
sc.tl.pca(pp_data, n_comps=9)

In [ ]:
sc.pl.pca(pp_data, color=['compound_name', 'CONDITION','MYCOND'], ncols=1, show=True, size=300)

In [ ]:
dc.get_metadata_associations(
    pp_data,
    obs_keys = ['compound_name', 'CONDITION', 'treatment_id', 'psbulk_n_cells', 'psbulk_counts'], #metadata columns to associate to PCs
    obsm_key='X_pca',  # where the PCs are stored
    uns_key='pca_anova',  # where the results are stored
    inplace=True
)

In [ ]:
plt.figure(figsize=(7,10))
ax, legend_axes = dc.plot_associations(
    pp_data,
    uns_key='pca_anova',  # summary statistics from the anova tests
    obsm_key='X_pca',  # where the PCs are stored
    stat_col='p_adj',  # which summary statistic to plot
    obs_annotation_cols = ['compound_name', 'CONDITION','treatment_id'], # which sample annotations to plot
    titles=['Adjusted p-values from ANOVA', 'Principle component scores']
)
plt.show()


In [ ]:
mypairs

In [ ]:
# Obtain genes that pass the thresholds
genes = dc.filter_by_expr(pdata, group='CONDITION', min_count=10, min_total_count=15)

# Filter by these genes
pdata = pdata[:, genes].copy()
pdata

In [ ]:
pdata.obs['MYCOND']

In [ ]:
pdatalist={}
ddslist={}
stat_reslist={}
results_dflist={}
for i in range(len(mypairs)):
    pdatalist[mypairs[i][0]+"_vs_"+mypairs[i][1]]=pdata[pdata.obs['MYCOND'].isin([mypairs[i][0], mypairs[i][1]])].copy()
    dds = DeseqDataSet(
            adata=pdatalist[mypairs[i][0]+"_vs_"+mypairs[i][1]],
            design_factors='MYCOND',
            ref_level=['MYCOND',mypairs[i][1]],
            refit_cooks=True)

    dds.deseq2()
    stat_res = DeseqStats(dds, contrast=['MYCOND',mypairs[i][0],mypairs[i][1]], n_cpus=8)
    stat_res.summary()
    stat_res.lfc_shrink(coeff='MYCOND_'+mypairs[i][0]+"_vs_"+mypairs[i][1])    
    results_dflist[mypairs[i][0]+"_vs_"+mypairs[i][1]] = stat_res.results_df
    ddslist[mypairs[i][0]+"_vs_"+mypairs[i][1]]=dds
    stat_reslist[mypairs[i][0]+"_vs_"+mypairs[i][1]]=stat_res
        

In [ ]:
#### Read the AAV processing factors list ####
#aavproc=pd.read_csv('Factors_AAV_processing_complete.csv', header=None)

In [ ]:
#goi=['Cxcl13','Cxcl10','Cxcl9','Ltbr','Fap','Cd19','Ms4a1','Cd8a','Gzmb','Ifng','Glycam1','Madcam1',
#     'Meox2','Cd274','Pdcd1']

In [ ]:
aavproc=goi.copy() #list(aavproc[0])

#### Overview of top DE genes per condition ####

In [ ]:
denrs={}
upslist={}
downslist={}
mymeans={}
for mycomp in results_dflist.keys():
    print(mycomp)
    dc.plot_volcano_df(results_dflist[mycomp], x='log2FoldChange', y='padj', top=20,  save='Volcano-DE-'+mysubset+"_"+mycomp+'.pdf')
    results_dflist[mycomp].to_csv(results_folder_out + mysubset+"_"+mycomp+'_Results.csv', index=True, index_label='GeneName')

    
    upslist[mycomp]=results_dflist[mycomp].loc[(results_dflist[mycomp].padj<0.2)&(results_dflist[mycomp].log2FoldChange>=np.log2(1.5)),:]
    downslist[mycomp]=results_dflist[mycomp].loc[(results_dflist[mycomp].padj<0.2)&(results_dflist[mycomp].log2FoldChange<=-np.log2(1.5)),:]
    denrs[mycomp]=[len(upslist[mycomp].index),len(downslist[mycomp].index)]
    
    mymeans[mycomp]=results_dflist[mycomp]['baseMean']
    



In [ ]:
mymeans=pd.DataFrame(mymeans)

In [ ]:
denrs[mycomp]=pd.DataFrame(denrs)
denrs[mycomp].index=['UP','DN']
plt.rcParams["figure.figsize"] = (4,4)
ax=denrs[mycomp].transpose().plot.bar()
ax.set_ylabel("Nr. pairwise DE genes")


In [ ]:
aavproc_up=set('')
aavproc_dn=set('')
for key in upslist.keys():
    aavproc_up=aavproc_up.union(set(upslist[key].index).intersection(set(aavproc)))
    aavproc_dn=aavproc_dn.union(set(downslist[key].index).intersection(set(aavproc)))

In [ ]:
upslist

In [ ]:
set(upslist[key].index)

In [ ]:
results_dflist.keys()

In [ ]:
UP1=set(results_dflist[key].loc[results_dflist['T0_vs_T1']['log2FoldChange'].abs()>=2,:].index)

In [ ]:
UP1

In [ ]:
UP2=set(results_dflist[key].loc[results_dflist['T0_vs_T2']['log2FoldChange'].abs()>=2,:].index)
UP22=set(results_dflist[key].loc[results_dflist['T1_vs_T2']['log2FoldChange'].abs()>=2,:].index)

In [ ]:
UP2

In [ ]:
UP2both=UP2.intersection(UP22)

In [ ]:
UP2both

In [ ]:
UP3=set(results_dflist[key].loc[results_dflist['T0_vs_T3']['log2FoldChange'].abs()>=2,:].index)
UP33=set(results_dflist[key].loc[results_dflist['T2_vs_T3']['log2FoldChange'].abs()>=2,:].index)

In [ ]:
UP4=set(results_dflist[key].loc[results_dflist['T0_vs_T4']['log2FoldChange'].abs()>=2,:].index)
UP44=set(results_dflist[key].loc[results_dflist['T3_vs_T4']['log2FoldChange'].abs()>=2,:].index)

In [ ]:
UP5=set(results_dflist[key].loc[results_dflist['T0_vs_T5']['log2FoldChange'].abs()>=2,:].index)
UP55=set(results_dflist[key].loc[results_dflist['T4_vs_T5']['log2FoldChange'].abs()>=2,:].index)

In [ ]:
UP5both=UP5.intersection(UP55)

In [ ]:
UP5

In [ ]:
UP5both

In [ ]:
UP3

In [ ]:
UP4

In [ ]:
UP3both=UP33.intersection(UP3)
UP4both=UP44.intersection(UP4)

In [ ]:
#### Select the genes with an average expression above a certain cutoff ####

In [ ]:
allde=list(UP1.union(UP2).union(UP3).union(UP4).union(UP5).union(UP22).union(UP33).union(UP44).union(UP55))
strde=list(UP2both.union(UP5).union(UP3both).union(UP4both))

In [ ]:
latede=list(list(UP3.union(UP5).union(UP4).union(UP44).union(UP55)))

In [ ]:
alldehigh=set(mymeans.loc[mymeans.mean(axis=1)>=100].index).intersection(allde)

In [ ]:
len(alldehigh)

In [ ]:
len(allde)

In [ ]:
len(strde)

In [ ]:
strde

In [ ]:
goiide=pd.DataFrame()
datade=pd.DataFrame()
datadeall=pd.DataFrame()
datadeh=pd.DataFrame()
for key in upslist.keys():
    tmp=results_dflist[key].loc[list(aavproc_up.union(aavproc_dn)),['log2FoldChange','padj']]
    tmp['cond']=key
    goiide=pd.concat([goiide,tmp], axis=0, join='outer', ignore_index=False,)
    
    tmp=results_dflist[key].loc[list(allde),['log2FoldChange','padj']]
    tmp['cond']=key
    datade=pd.concat([datade,tmp], axis=0, join='outer', ignore_index=False,)
    
    tmp=results_dflist[key].loc[:,['log2FoldChange','padj']]
    tmp['cond']=key
    datadeall=pd.concat([datadeall,tmp], axis=0, join='outer', ignore_index=False,)

    tmp=results_dflist[key].loc[list(alldehigh),['log2FoldChange','padj']]
    tmp['cond']=key
    datadeh=pd.concat([datadeh,tmp], axis=0, join='outer', ignore_index=False,)

In [ ]:
aavproc_up

In [ ]:
goiide.to_csv(results_folder_out + "Goi_processing_complete_"+mysubset+"_all_SigDE.csv", index=True, index_label='GeneName')
datade.to_csv(results_folder_out + "TopUPs_"+mysubset+"_all_SigDE.csv", index=True, index_label='GeneName')


In [ ]:
goiide

In [ ]:
#goiide.loc['Glycam1',:]
len(goiide)

In [ ]:
dplot3=(-1)*goiide.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T0_vs_T2','T0_vs_T3',
                                                                       'T0_vs_T4','T0_vs_T5']]
dplot3.columns=['T1_T0','T2_T0','T3_T0','T4_T0','T5_T0']

dplot3v=(-1)*goiide.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T1_vs_T2','T2_vs_T3',
                                                                       'T3_vs_T4','T4_vs_T5']]
dplot3v.columns=['T1_T0','T2_T1','T3_T2','T4_T3','T5_T4']



In [ ]:
if len(goiide)>4:
    sns.set(font_scale=0.8) 
    sns.clustermap(dplot3, cmap="vlag", figsize=(3, 7),center=0,col_cluster=False)

In [ ]:
if len(goiide)>4:
    sns.set(font_scale=0.8) 
    sns.clustermap(dplot3v, cmap="vlag", figsize=(3, 7),center=0,col_cluster=False)

In [ ]:
datade.pivot(columns="cond", values="log2FoldChange").fillna(0)

In [ ]:
datadeh

In [ ]:
dplot2=(-1)*datadeh.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T1_vs_T2','T2_vs_T3',
                                                                       'T3_vs_T4','T4_vs_T5']]
dplot2.columns=['T1_T0','T2_T1','T3_T2','T4_T3','T5_T4']
sns.set(font_scale=0.7) 
sns.clustermap(dplot2,figsize=(3, 11),col_cluster=False, 
               cmap="vlag", center=0)

In [ ]:
dplot2v=(-1)*datadeh.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T0_vs_T2','T0_vs_T3',
                                                                       'T0_vs_T4','T0_vs_T5']]
dplot2v.columns=['T1_T0','T2_T0','T3_T0','T4_T0','T5_T0']
sns.set(font_scale=0.7) 
sns.clustermap(dplot2v,figsize=(3, 11),col_cluster=False, 
               cmap="vlag", center=0)

In [ ]:
dplot2=(-1)*datade.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T1_vs_T2','T2_vs_T3',
                                                                       'T3_vs_T4','T4_vs_T5']]
dplot2.columns=['T1_T0','T2_T1','T3_T2','T4_T3','T5_T4']
sns.set(font_scale=0.8) 
sns.clustermap(dplot2.loc[list(strde),:],figsize=(3, 9),col_cluster=False, 
               cmap="vlag", center=0)

In [ ]:
dplot2v=(-1)*datade.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T0_vs_T2','T0_vs_T3',
                                                                       'T0_vs_T4','T0_vs_T5']]
dplot2v.columns=['T1_T0','T2_T0','T3_T0','T4_T0','T5_T0']
sns.set(font_scale=0.8) 
sns.clustermap(dplot2v.loc[list(strde),:],figsize=(3, 9),col_cluster=False, 
               cmap="vlag", center=0)

In [ ]:
dplot2=(-1)*datade.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T1_vs_T2','T2_vs_T3',
                                                                       'T3_vs_T4','T4_vs_T5']]
dplot2.columns=['T1_T0','T2_T1','T3_T2','T4_T3','T5_T4']
sns.set(font_scale=0.8) 
sns.clustermap(dplot2.loc[list(UP3.union(UP5).union(UP4).union(UP44).union(UP55)),:],figsize=(3, 8),col_cluster=False, 
               cmap="vlag", center=0)

In [ ]:
dplot2v=(-1)*datade.pivot(columns="cond", values="log2FoldChange").fillna(0).loc[:,['T0_vs_T1','T0_vs_T2','T0_vs_T3',
                                                                       'T0_vs_T4','T0_vs_T5']]
dplot2v.columns=['T1_T0','T2_T0','T3_T0','T4_T0','T5_T0']
sns.set(font_scale=0.8) 
sns.clustermap(dplot2v.loc[list(list(UP3.union(UP5).union(UP4).union(UP44).union(UP55))),:],figsize=(3, 8),col_cluster=False, 
               cmap="vlag", center=0)

In [ ]:
sns.clustermap(datade.pivot(columns="cond", values="log2FoldChange").fillna(0),figsize=(5, 11), 
               cmap="vlag", center=0)

In [ ]:
goidesorted=list(results_dflist['T0_vs_T2'].loc[list(aavproc_up.union(aavproc_dn)),:].sort_values(by='log2FoldChange', ascending=True).index)

In [ ]:
list(aavproc_up.union(aavproc_dn))

In [ ]:
datadesorted=list(results_dflist['T0_vs_T2'].loc[list(set(datadeh.index)),:].sort_values(by='log2FoldChange', ascending=True).index)

In [ ]:
cdatak.obs['sample_treat']=cdatak.obs['individual_id'].astype(str)+"_"+cdatak.obs['MYCOND'].str.slice(0, 10)


In [ ]:
cdata.obs['sample_treat']=cdata.obs['individual_id'].astype(str)+"_"+cdata.obs['MYCOND'].str.slice(0, 10)

set(cdata.obs['sample_treat'])


In [ ]:

korder=['4701_50',
 '4708_96',
 '4789_80',
 '4852_91',
 'B1',
 'B2',
 'B11',
 'B33',
 'B37',
 'B6',
 'B3',
 'B13',
 'B7',
 'B29',
 'B20',
 'B24',
 'B12',
 'B25',
 'B23',
 'B26',
 'B17',
 'B35',
 'B39',
 'B31'] # 'I201_MaleAAV2-C',

#DElist['FemaleAAV2-CMV-GFP-FemaleUntreated']


#### Genes upregulated in at least one condition 

In [ ]:
sc.settings.set_figure_params()

In [ ]:
if (len(goidesorted)>0):
    sc.pl.matrixplot(cdatak,goidesorted, groupby='individual_id', standard_scale='var',
                 categories_order=korder)
else:
        sc.pl.matrixplot(cdatak,goi, groupby='individual_id', standard_scale='var',
                 categories_order=korder)

In [ ]:
sc.pl.matrixplot(cdatak,datadesorted, groupby='individual_id', standard_scale='var',
                 categories_order=korder)

In [ ]:
if (len(goidesorted)>0):
    sc.pl.matrixplot(cdatak,goidesorted, groupby='MYCOND', standard_scale='var')
else:
    sc.pl.matrixplot(cdatak,goi, groupby='MYCOND', standard_scale='var')

In [ ]:
sc.pl.matrixplot(cdatak,datadesorted, groupby='MYCOND', standard_scale='var')

In [ ]:
sc.pl.dotplot(cdatak,var_names=datadesorted, groupby='MYCOND')

In [ ]:
sc.pl.dotplot(cdatak,var_names=datadesorted, groupby='MYCOND', dot_max=0.3, vmax=2)

#### Pathway and TF enrichment analysis 

In [ ]:
mypval=0.05
myfc=np.log2(1.5)
mypvalstr=str(mypval).replace('.','')[1:3]
myfcstr=str(myfc).replace('.','')[0:2]



# Retrieve CollecTRI gene regulatory network
collectri = dc.get_collectri(organism='mouse', split_complexes=False)
progeny_mouse = pd.read_csv("model_progeny500_mouse_decoupleR.csv")
msigdb_mouse = pd.read_csv("msigdb_mouse_hallmark.csv")
msigdb_mouse = msigdb_mouse[~msigdb_mouse.duplicated(['gene_symbol', 'gs_name'])]
msigdb_mouse.loc[:, 'geneset'] = [name.split('HALLMARK_')[1] for name in msigdb_mouse['gs_name']]

import gseapy

dbs=['KEGG_2021_Human','KEGG_2019_Mouse','GO_Biological_Process_2023','Reactome_2022','Human_Gene_Atlas',
     'WikiPathways_2019_Mouse','NCI-Nature_2016',
     'ChEA_2022','CellMarker_Augmented_2021','Azimuth_Cell_Types_2021',
     'COVID-19_Related_Gene_Sets_2021','MSigDB_Hallmark_2020','TG_GATES_2020']
cdata.var['MeanExpr']=cdata.X.mean(axis=0).tolist()[0]
if not os.path.exists(results_folder_out): os.mkdir(results_folder_out)
if not os.path.exists(results_folder_out+'enrichr/'): os.mkdir(results_folder_out+'enrichr/')


In [ ]:
tf_pvalslist={}
tf_actslist={}
pathway_actslist={}
pathway_pvalslist={}
enr_pvalslist={}

for mycomp in results_dflist.keys():
    print(mycomp)
    results_df=results_dflist[mycomp]
    mat = results_df[['stat']].T.rename(index={'stat': mycomp})
    mat.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
    
    # Infer pathway activities with ulm
    tf_acts, tf_pvals = dc.run_ulm(mat=mat, net=collectri,min_n=5)
    dc.plot_barplot(tf_acts, mycomp, top=25, vertical=False,save='TF_acts_'+mysubset+"_"+mycomp+'.pdf')
    tf_actslist[mycomp]=tf_acts
    tf_pvalslist[mycomp]=tf_pvals
    
    # Extract logFCs and pvals
    logFCs = results_df[['log2FoldChange']].T.rename(index={'log2FoldChange': 'All'})
    pvals = results_df[['padj']].T.rename(index={'padj': 'All'})
    #dc.plot_volcano(logFCs, pvals, 'All', name='Hdac9', net=collectri, top=10, sign_thr=0.05, lFCs_thr=0.5,
    #                save='TF_acts_'+mysubset+"_"+mycomp+'.pdf')

    
    pathway_acts, pathway_pvals = dc.run_mlm(mat=mat, net=progeny_mouse)
    dc.plot_barplot(pathway_acts, mycomp, top=25, vertical=False, save='TF_acts_'+mysubset+"_"+mycomp+'.pdf')
    pathway_actslist[mycomp]=pathway_acts
    pathway_pvalslist[mycomp]=pathway_pvals
    
    # Infer enrichment with ora using significant deg
    top_genes = results_df[results_df['padj'] < 0.05]

    # Run ora
    enr_pvals = dc.get_gsea_df(
        df=results_df,
        stat = 'stat',
        net=msigdb_mouse,
        source='geneset',
        target='gene_symbol'
    )

    enr_pvals.sort_values('NES', ascending=False)
    enr_pvals_filtered = enr_pvals[enr_pvals['FDR p-value'] < 0.05]
    if (len(enr_pvals_filtered)>=1):
        dc.plot_dotplot(enr_pvals_filtered, x='NES', y = 'Term', s='NES', c = 'FDR p-value', scale = 0.5, figsize=(7,10))
        enr_pvals_filtered.to_csv(results_folder_out + 'ORA_Hallmarks_'+mysubset+"_"+mycomp+'.tsv', index=True, index_label='GeneName')
    enr_pvalslist[mycomp]=enr_pvals
       
        
    for j in dbs:
        try:
            gseapy.enrichr(gene_list=list(upslist[mycond].index), organism='mouse', 
                       gene_sets=j, background=list(cdata.var.loc[cdata.var['MeanExpr']>0.01]['SYMBOL']),
                               cutoff=mypval, format='png',outdir=results_folder_out+'enrichr/'+myset+'-DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mycomp+'/')
        except:
            print('No enrichr terms for '+j+ ' and UP')
        try:
            gseapy.enrichr(gene_list=list(downslist[mycond].index), organism='mouse', 
                       gene_sets=j, background=list(cdata.var.loc[cdata.var['MeanExpr']>0.01]['SYMBOL']),
                               cutoff=mypval, format='png',outdir=results_folder_out+'enrichr/'+myset+'-DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-DN-'+mycomp+'/')
        except:
            print('No enrichr terms for '+j+ ' and DN')

In [ ]:
tfcut=2.5
sigcut=0.5

In [ ]:
pathway_actslist.keys()

In [ ]:
p1=set(pathway_actslist['T0_vs_T1'].loc[: , (pathway_actslist['T0_vs_T1'].abs() >=1).any()].columns)
p2=set(pathway_actslist['T0_vs_T2'].loc[: , (pathway_actslist['T0_vs_T2'].abs() >=1).any()].columns)
p3=set(pathway_actslist['T0_vs_T3'].loc[: , (pathway_actslist['T0_vs_T3'].abs() >=1).any()].columns)
p4=set(pathway_actslist['T0_vs_T4'].loc[: , (pathway_actslist['T0_vs_T4'].abs() >=1).any()].columns)
p5=set(pathway_actslist['T0_vs_T5'].loc[: , (pathway_actslist['T0_vs_T5'].abs() >=1).any()].columns)

In [ ]:
p1

In [ ]:
p2

In [ ]:
p3

In [ ]:
p4

In [ ]:
p5

In [ ]:

tf1=set(tf_actslist['T0_vs_T1'].loc[: , (tf_actslist['T0_vs_T1'].abs() >=tfcut).any()].columns)
tf2=set(tf_actslist['T0_vs_T2'].loc[: , (tf_actslist['T0_vs_T2'].abs() >=tfcut).any()].columns)
tf3=set(tf_actslist['T0_vs_T3'].loc[: , (tf_actslist['T0_vs_T3'].abs() >=tfcut).any()].columns)
tf4=set(tf_actslist['T0_vs_T4'].loc[: , (tf_actslist['T0_vs_T4'].abs() >=tfcut).any()].columns)
tf5=set(tf_actslist['T0_vs_T5'].loc[: , (tf_actslist['T0_vs_T5'].abs() >=tfcut).any()].columns)

In [ ]:
tf1

In [ ]:
nescut=1

In [ ]:
#sv=set(enr_pvalslist['V3X_vs_V24H'].loc[(enr_pvalslist['V3X_vs_V24H']['NES'].abs() >=nescut),"Term"])

s1=set(enr_pvalslist['T0_vs_T1'].loc[(enr_pvalslist['T0_vs_T1']['NES'].abs() >=nescut),"Term"])
s2=set(enr_pvalslist['T0_vs_T2'].loc[(enr_pvalslist['T0_vs_T2']['NES'].abs() >=nescut),"Term"])
s3=set(enr_pvalslist['T0_vs_T3'].loc[ (enr_pvalslist['T0_vs_T3']['NES'].abs() >=nescut),"Term"])
s4=set(enr_pvalslist['T0_vs_T4'].loc[(enr_pvalslist['T0_vs_T4']['NES'].abs() >=nescut),"Term"])
s5=set(enr_pvalslist['T0_vs_T5'].loc[(enr_pvalslist['T0_vs_T5']['NES'].abs() >=nescut),"Term"])


In [ ]:
allp=p1.union(p2).union(p3).union(p4).union(p5)
allt=tf1.union(tf2).union(tf3).union(tf4).union(tf5)
alls=s1.union(s2).union(s3).union(s4).union(s5)

In [ ]:
patde=pd.DataFrame()
sigde=pd.DataFrame()
tfde=pd.DataFrame()

for key in upslist.keys():
    tmp=pathway_actslist[key].loc[:,list(allp)]
    tmp['cond']=key
    patde=pd.concat([patde,tmp], axis=0, join='outer', ignore_index=False,)
    
    tmp=tf_actslist[key].loc[:,list(allt)]
    tmp['cond']=key
    tfde=pd.concat([tfde,tmp], axis=0, join='outer', ignore_index=False,)
   
    tmp=enr_pvalslist[key].loc[enr_pvalslist[key]['Term'].isin(list(alls)),:]
    tmp['cond']=key
    sigde=pd.concat([sigde,tmp], axis=0, join='outer', ignore_index=False,)


In [ ]:
sigde

In [ ]:
patde.to_csv(results_folder_out + "TopUPs_Pathways_"+mysubset+"_all.csv", index=False)
patde=patde.drop(columns='cond').transpose()

In [ ]:
tfde.to_csv(results_folder_out + "TopUPs_TFs_"+mysubset+"_all.csv", index=False)
tfde=tfde.drop(columns='cond').transpose()

In [ ]:
sigde.to_csv(results_folder_out + "TopUPs_Signatures_"+mysubset+"_all.csv", index=False)
#sigde=tfde.drop(columns='cond').transpose()

In [ ]:
sigde=sigde.pivot(columns="cond", values="NES", index='Term')

In [ ]:
g=sns.clustermap(sigde,figsize=(6, 12), cmap="vlag", center=0)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0) # For x axis

In [ ]:

g=sns.clustermap(patde,figsize=(4, 6), cmap="vlag", center=0)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0) # For x axis

In [ ]:

g=sns.clustermap(tfde,figsize=(4, 9), cmap="vlag", center=0)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0) # For x axis

In [ ]:
#dc.plot_targets(results_df, stat='stat', source_name='TGFb', net=progeny_mouse, top=15)

#dc.plot_targets(results_df, stat='stat', source_name='MAPK', net=progeny_mouse, top=15)

##dc.plot_targets(results_df, stat='stat', source_name='PI3K', net=progeny_mouse, top=15)

## Functional enrichment of biological terms


In [ ]:
#infolder='/projects/site/pred/scseq/projects/PS-10732_400045_SNS_of_AAV_livers_8050P21/fulldata/'
#myaav=pd.read_csv(infolder+'Factors_AAV_processing_final.csv',sep='\t', header=None)


In [ ]:
#myaav=list(set(list(myaav[0])))

#### Genes of interest, DE in both approaches (AAV vs. C)

In [ ]:
sc.pl.matrixplot(cdata,list(set(goi)), groupby='MYCOND', standard_scale='var')

In [ ]:
pubg=['Spp1', 'Mmp2', 'Tgfb1', 'Apoe', 'Ctsb', 'H2-Aa', 'H2-Ab1', 'Cxcl9', 'Cxcl10']

In [ ]:
sc.pl.matrixplot(cdata,list(set(datadesorted)), groupby='MYCOND', standard_scale='var')

In [ ]:
sc.pl.umap(cdata,color='celltype4')

In [ ]:
def getAverageGeneExpression(sdata, myg, sample):
    # Table with average gene expression per condition
    obs = sdata.raw[:,myg].X.toarray()
    obs = pd.DataFrame(obs, columns=myg, index=sdata.obs[sample])
    average_obs = obs.groupby(level=0).mean()

    print("Average expression per gene and sample")
    display(average_obs)
    return average_obs

In [ ]:
goi=list(set(goi))

In [ ]:
average_obs = getAverageGeneExpression(cdatak, pubg, 'individual_id')

In [ ]:
average_obs=average_obs.loc[:,(average_obs!=0).any(axis=0)]

In [ ]:
average_obs

In [ ]:
import math

In [ ]:
sdata=cdatak.copy()
myg=goi
tr='treatment_id'
sample='individual_id'

In [ ]:
mypalette={"baseline":"black", "CEA-TCB": "red"}

In [ ]:
def plotBoxplot(sdata, average_obs, myg, sample, tr):
    tmp=sdata.obs[[sample,tr,'compound_name']].drop_duplicates().merge(average_obs, on=sample, how="left")

    max_cols = 5
    rows = math.ceil(len(myg)/max_cols)
    col = min(max_cols, len(myg))

    fig, axes = plt.subplots(rows, col, figsize=(5*col, 4*rows), gridspec_kw={'wspace': 0.55, 'hspace': 0.4, 'left': 0.25})
    plt.subplots_adjust(left=0.1, right=0.98, top=0.82, bottom=0.1)
    if len(myg) == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    for i, gene in enumerate(myg):
        ax=sns.boxplot(data=tmp, y=gene, x=tr, fliersize=0, ax=axes[i], hue='compound_name', palette=mypalette)
        sns.stripplot(data=tmp, y=gene, x=tr, dodge=True, color="black", jitter=0.2, 
                              size=5, ax=ax, hue='compound_name')
        dump = ax.set_xticklabels([t.get_text() for t in ax.get_xticklabels()], rotation = 0)
        handles, labels = ax.get_legend_handles_labels()
        subset=int(len(handles)/2) # Only legeng for boxplot (half of the handles), not for the dots
        ax.legend(handles[0:subset], labels[0:subset], fontsize='12', title_fontsize='5')
        
    #plt.savefig(figdir+'GeneExpression-pubgenes-'+mysubset+'.pdf')
    #plt.savefig(figdir+'GeneExpression-pubgenes-'+mysubset+'.svg')

In [ ]:
#goi=list(set(goi).intersection(set(average_obs.columns)))

In [ ]:
list(set(goi).intersection(set(average_obs.columns)))

In [ ]:
figdir=os.path.join(root_path, 'analyzed', analysis_name+'/figures/')

In [ ]:
plotBoxplot(cdatak, average_obs, pubg, 'individual_id', 'treatment_id')


In [ ]:
average_obs = getAverageGeneExpression(cdatak, goi, 'individual_id')

In [ ]:
plotBoxplot(cdatak, average_obs, list(set(goi).intersection(set(average_obs.columns))), 'individual_id', 'treatment_id')

In [ ]:
pubg

In [ ]:
dg=['Isg15', 'Ifit1', 'Ifit3', 'Ido1', 'Cxcl9', 'Cxcl10', 'H2-Aa', 'H2-Ab1', 
    'Spp1', 'Cd163', 'Vsig4', 'Fn1', 'Gpnmb', 'Mrc1', 'Trem2', 'Lgmn', 'Msr1', 'Ccl4', 'Il6', 'Egr4', 'Cd274', 
    'Ctsd', 'Ctss']

In [ ]:

sc.settings.set_figure_params()

In [ ]:
sc.pl.dotplot(cdatak,var_names=dg, groupby='treatment_id',  save='-treatment_id-'+mysubset+'.nonorm.svg')

In [ ]:
sc.pl.matrixplot(cdatak,var_names=dg, groupby='treatment_id', standard_scale='var', save='-treatment_id-'+mysubset+'.nonorm.svg')

In [ ]:
pubg=['Cxcl9', 'Cxcl10','Isg15','Ifit1','Ifit3','Ido1','Cd274','H2-Aa', 'H2-Ab1',  'Mmp2',  'Apoe', 'Ctsb','Trem2','Lgmn','Egr4', 'Tgfb1','Spp1']

In [ ]:
sc.pl.matrixplot(cdatak,var_names=pubg, groupby='treatment_id', standard_scale='var', save='-treatment_id-'+mysubset+'.pubg.nonorm.svg')

In [ ]:
sc.pl.dotplot(cdatak,var_names=pubg, groupby='treatment_id',  
              save='-treatment_id-'+mysubset+'.pubg.nonorm.svg')

In [ ]:
sns.set(font_scale=0.8)
a=sns.clustermap(average_obs.loc[korder,:], col_cluster=True, row_cluster=False, figsize=(12,7),
                 cmap='viridis', metric='correlation', standard_scale=1) #row_colors=mycol, vmax=0.8

In [ ]:
#sns.set(font_scale=0.8)
#a=sns.clustermap(average_obs.loc[korder,goiinf], col_cluster=True, row_cluster=False, figsize=(10,6),
#                 cmap='viridis', metric='correlation', standard_scale=1) #row_colors=mycol, vmax=0.8

In [ ]:
#sns.set(font_scale=0.8)
#a=sns.clustermap(average_obs.loc[korder,goirep], col_cluster=True, row_cluster=False, figsize=(10,6),
#                 cmap='viridis', metric='correlation', standard_scale=1) #row_colors=mycol, vmax=0.8

In [ ]:
average_obs = getAverageGeneExpression(cdatak, goidesorted, 'individual_id')


In [ ]:

sns.set(font_scale=0.8)
a=sns.clustermap(average_obs.loc[korder,:], col_cluster=True, row_cluster=False, figsize=(9,5),
                 cmap='viridis', metric='correlation', standard_scale=1) #row_colors=mycol, vmax=0.8

sc.settings.set_figure_params()





In [ ]:
plotBoxplot(cdatak, average_obs, goidesorted, 'individual_id', 'treatment_id')

In [ ]:
len(alldehigh)

In [ ]:
#goichoice=['Atacayos', 'Fam71a', Ifitm10, Npnt] 

In [ ]:
#average_obs = getAverageGeneExpression(cdatak, list(alldehigh), 'individual_id')

average_obs = getAverageGeneExpression(cdatak, list(allde), 'individual_id')

In [ ]:
sns.set(font_scale=0.8)
a=sns.clustermap(average_obs.loc[korder,:], col_cluster=True, row_cluster=False, figsize=(13,6),
                 cmap='viridis', metric='correlation', standard_scale=1) #row_colors=mycol, vmax=0.8

In [ ]:
sc.settings.set_figure_params()

In [ ]:
plotBoxplot(cdatak, average_obs, list(alldehigh), 'individual_id', 'treatment_id')

In [ ]:
average_obs = getAverageGeneExpression(cdatak, latede, 'individual_id')

In [ ]:
sns.set(font_scale=0.8)
a=sns.clustermap(average_obs.loc[korder,:], col_cluster=True, row_cluster=False, figsize=(12,6),
                 cmap='viridis', metric='correlation', standard_scale=1) #row_colors=mycol, vmax=0.8

In [ ]:
sc.settings.set_figure_params()


In [ ]:

plotBoxplot(cdatak, average_obs, latede, 'individual_id', 'treatment_id')

In [ ]:
#sc.pl.umap(adata, color=['Ceacam1'])

In [ ]:
! jupyter nbconvert --to html DEAnalysis-Decoupler.ipynb